In [1]:
from itertools import combinations_with_replacement
from ehrhart_quasi_polynomia import secondary_fan, ehrhart_quasi_polynomia, create_polyhedron_from_matrix

In [22]:
from sage.functions.other import factorial
from sage.modules.free_module_element import free_module_element
from sage.rings.polynomial.polynomial_ring_constructor import PolynomialRing

In [3]:
from inspect import *

In [58]:
def generate_cone_points(cone, number):
    dimension = cone._dim
    amb_dim = cone._lattice._Module_free_ambient__degree

    # print(f"cone of dim = {dimension} in {amb_dim}d ambient space")

    if dimension == 0:
        return [(0,)*amb_dim], 0

    vectors = [free_module_element(list(ray)) for ray in cone._rays]

    points = []
    points_len = 0
    index = 1
    while points_len <= number:
        new_points = [sum(combi) for combi in combinations_with_replacement(vectors, index)]
        points += new_points
        index += 1
        points_len += len(new_points)
    return points[points_len-number:]

In [5]:
A = Matrix([[-1, 0], [0, -1], [1, 1], [0, 1]])
sec_fan = secondary_fan(A)
cones = sec_fan["CONES"]
cones

[0-d cone in 1-d lattice N,
 1-d cone in 4-d lattice N,
 1-d cone in 4-d lattice N,
 1-d cone in 4-d lattice N,
 2-d cone in 4-d lattice N,
 2-d cone in 4-d lattice N]

In [6]:
R = PolynomialRing(QQ, "x", 4)
R.gens()

(x0, x1, x2, x3)

In [7]:
f = lambda w, x, y, z: w**2 + x*y + z
R.interpolation(2, f)

x0^2 + x1*x2 + x3

In [8]:
def get_terms_of_order(polynomials, order):
    return [poly.coefficients()[order].constants()[0] if poly.degree() >= order else 0 for poly in polynomials]

In [9]:
def get_term_dict(A_matrix, points):
    polynomials = []
    for point in points:
        polytope = create_polyhedron_from_matrix(A_matrix, point)
        ehr_poly = ehrhart_quasi_polynomia(polytope.Vrepresentation(), True)
        polynomials.append(ehr_poly)

    max_degree = max(poly.degree() for poly in polynomials)
    terms = {d: get_terms_of_order(polynomials, d) for d in range(max_degree + 1)}
    return terms

In [55]:
def _compute_piecewise(A_matrix, sec_fan):
    num_variables = sec_fan["DIM"]
    max_degree = sec_fan["LINEALITY_DIM"]

    R = PolynomialRing(QQ, "x", 4)
    S = PolynomialRing(R, "k")
    k = S.gen()

    cone_polys = []
    for cone in sec_fan["CONES"][1:]:
        estimated_period = 1
        needed_points = factorial(num_variables + max_degree)//( factorial(num_variables)*factorial(max_degree) )
        cone_points = generate_cone_points(cone, needed_points)
        term_dict = get_term_dict(A_matrix, cone_points)

        cone_poly = 0
        for degree, terms in term_dict.items():
            try:
                cone_poly += R.interpolation(max_degree, cone_points, terms)*k**degree
            except:
                return max_degree, cone_points, terms
        print(cone_poly)
        cone_polys.append(cone_poly)
    return cone_polys
        

In [59]:
error = _compute_piecewise(A, sec_fan)

1
21/8*x3^2*k^2 + 13/4*x3*k + 1
9/2*x3^2*k^2 - 11/2*x3*k + 1


In [60]:
error

(2,
 [(-3, -3, 6, 9),
  (-1, -1, 7, 8),
  (1, 1, 8, 7),
  (3, 3, 9, 6),
  (-4, -4, 8, 12),
  (-2, -2, 9, 11),
  (0, 0, 10, 10),
  (2, 2, 11, 9),
  (4, 4, 12, 8),
  (-5, -5, 10, 15),
  (-3, -3, 11, 14),
  (-1, -1, 12, 13),
  (1, 1, 13, 12),
  (3, 3, 14, 11),
  (5, 5, 15, 10)],
 [0, 15/2, 14, 39/2, 0, 15/2, 15, 41/2, 26, 0, 15/2, 15, 43/2, 27, 65/2])

In [51]:
from collections.abc import Iterable
def interpolation(self, bound, *args):
        # get ring and number of variables
        R = self.base_ring()
        n = self.ngens()

        # we only run the algorithm over fields
        if not R.is_field():
            raise TypeError(f'The base ring {R} is not a field.')

        # helper function to sample "num_samples" elements from R
        def sample_points(num_samples):
            try:
                samples = list(itertools.islice(R, num_samples))
                if len(samples) < num_samples:
                    raise ValueError(f'Could not sample {num_samples} different elements of {R}.')
            except NotImplementedError:
                if R.characteristic() == 0 or R.characteristic() >= num_samples:
                    samples = [R(k) for k in range(num_samples)]
                else:
                    raise NotImplementedError(f'Could not sample {num_samples} different elements of {R}.')

            return samples

        # set points and values
        if len(args) == 2:
            points, values = args
        else:
            F, = args

            if isinstance(bound, Iterable):
                R_points = sample_points(max(bound) + 1)
                points = list(itertools.product(*[R_points[:bound[i] + 1] for i in range(n)]))
            else:
                points = list(itertools.combinations_with_replacement(sample_points(bound + 1), n))

            values = [F(*x) for x in points]

        # find all possibly appearing exponents
        if isinstance(bound, Iterable):
            exponents_space = list(itertools.product(*(range(bound[i] + 1) for i in range(n))))
        else:
            exponents_space = []
            for entry in itertools.combinations_with_replacement(range(bound + 1), n):
                exponents_space.append([entry[0]] + [entry[i] - entry[i - 1] for i in range(1, n)])
    
        # build matrix
        M = matrix.zero(R, 0, len(points))
        for exponents in exponents_space:
            M = M.stack(vector(R, [self.monomial(*exponents)(*x) for x in points]))

        # solve for coefficients and construct polynomial
        try:
            coeff = M.solve_left(vector(R, values))
        except ValueError:
            raise ValueError('Could not find a solution.')
        solution = sum(coeff[i] * self.monomial(*exponents_space[i]) for i in range(len(exponents_space)))

        return solution

In [19]:
R = PolynomialRing(QQ, "x", 4)

In [30]:
error

(2,
 [(-3, -3, 6, 9),
  (-1, -1, 7, 8),
  (1, 1, 8, 7),
  (3, 3, 9, 6),
  (-4, -4, 8, 12),
  (-2, -2, 9, 11),
  (0, 0, 10, 10),
  (2, 2, 11, 9),
  (4, 4, 12, 8),
  (-5, -5, 10, 15),
  (-3, -3, 11, 14),
  (-1, -1, 12, 13),
  (1, 1, 13, 12),
  (3, 3, 14, 11),
  (5, 5, 15, 10)],
 [0, 15/2, 14, 39/2, 0, 15/2, 15, 41/2, 26, 0, 15/2, 15, 43/2, 27, 65/2])

In [44]:
exponents_space

[[0, 0, 0, 0],
 [0, 0, 0, 1],
 [0, 0, 0, 2],
 [0, 0, 1, 0],
 [0, 0, 1, 1],
 [0, 0, 2, 0],
 [0, 1, 0, 0],
 [0, 1, 0, 1],
 [0, 1, 1, 0],
 [0, 2, 0, 0],
 [1, 0, 0, 0],
 [1, 0, 0, 1],
 [1, 0, 1, 0],
 [1, 1, 0, 0],
 [2, 0, 0, 0]]

In [47]:
M

[  1   1   1   1   1   1   1   1   1   1   1   1   1   1   1]
[  9   8   7   6  12  11  10   9   8  15  14  13  12  11  10]
[ 81  64  49  36 144 121 100  81  64 225 196 169 144 121 100]
[  6   7   8   9   8   9  10  11  12  10  11  12  13  14  15]
[ 54  56  56  54  96  99 100  99  96 150 154 156 156 154 150]
[ 36  49  64  81  64  81 100 121 144 100 121 144 169 196 225]
[ -3  -1   1   3  -4  -2   0   2   4  -5  -3  -1   1   3   5]
[-27  -8   7  18 -48 -22   0  18  32 -75 -42 -13  12  33  50]
[-18  -7   8  27 -32 -18   0  22  48 -50 -33 -12  13  42  75]
[  9   1   1   9  16   4   0   4  16  25   9   1   1   9  25]
[ -3  -1   1   3  -4  -2   0   2   4  -5  -3  -1   1   3   5]
[-27  -8   7  18 -48 -22   0  18  32 -75 -42 -13  12  33  50]
[-18  -7   8  27 -32 -18   0  22  48 -50 -33 -12  13  42  75]
[  9   1   1   9  16   4   0   4  16  25   9   1   1   9  25]
[  9   1   1   9  16   4   0   4  16  25   9   1   1   9  25]

In [61]:
interpolation(R, 2, error[1], error[2])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[9, 8, 7, 6, 12, 11, 10, 9, 8, 15, 14, 13, 12, 11, 10]
[81, 64, 49, 36, 144, 121, 100, 81, 64, 225, 196, 169, 144, 121, 100]
[6, 7, 8, 9, 8, 9, 10, 11, 12, 10, 11, 12, 13, 14, 15]
[54, 56, 56, 54, 96, 99, 100, 99, 96, 150, 154, 156, 156, 154, 150]
[36, 49, 64, 81, 64, 81, 100, 121, 144, 100, 121, 144, 169, 196, 225]
[-3, -1, 1, 3, -4, -2, 0, 2, 4, -5, -3, -1, 1, 3, 5]
[-27, -8, 7, 18, -48, -22, 0, 18, 32, -75, -42, -13, 12, 33, 50]
[-18, -7, 8, 27, -32, -18, 0, 22, 48, -50, -33, -12, 13, 42, 75]
[9, 1, 1, 9, 16, 4, 0, 4, 16, 25, 9, 1, 1, 9, 25]
[-3, -1, 1, 3, -4, -2, 0, 2, 4, -5, -3, -1, 1, 3, 5]
[-27, -8, 7, 18, -48, -22, 0, 18, 32, -75, -42, -13, 12, 33, 50]
[-18, -7, 8, 27, -32, -18, 0, 22, 48, -50, -33, -12, 13, 42, 75]
[9, 1, 1, 9, 16, 4, 0, 4, 16, 25, 9, 1, 1, 9, 25]
[9, 1, 1, 9, 16, 4, 0, 4, 16, 25, 9, 1, 1, 9, 25]


[  1   1   1   1   1   1   1   1   1   1   1   1   1   1   1]
[  9   8   7   6  12  11  10   9   8  15  14  13  12  11  10]
[ 81  64  49  36 144 121 100  81  64 225 196 169 144 121 100]
[  6   7   8   9   8   9  10  11  12  10  11  12  13  14  15]
[ 54  56  56  54  96  99 100  99  96 150 154 156 156 154 150]
[ 36  49  64  81  64  81 100 121 144 100 121 144 169 196 225]
[ -3  -1   1   3  -4  -2   0   2   4  -5  -3  -1   1   3   5]
[-27  -8   7  18 -48 -22   0  18  32 -75 -42 -13  12  33  50]
[-18  -7   8  27 -32 -18   0  22  48 -50 -33 -12  13  42  75]
[  9   1   1   9  16   4   0   4  16  25   9   1   1   9  25]
[ -3  -1   1   3  -4  -2   0   2   4  -5  -3  -1   1   3   5]
[-27  -8   7  18 -48 -22   0  18  32 -75 -42 -13  12  33  50]
[-18  -7   8  27 -32 -18   0  22  48 -50 -33 -12  13  42  75]
[  9   1   1   9  16   4   0   4  16  25   9   1   1   9  25]
[  9   1   1   9  16   4   0   4  16  25   9   1   1   9  25]